# Visualizing Data in Roman ASDF Files

***

## Imports
- *astropy.visualization.simple_norm* for automatically scaling image arrays
- *astropy.coordinates.SkyCoord* for creating Python objects containing sky coordinate transforms
- *matplotlib.pyplot* for creating static image previews
- *jdaviz.Imviz* for interactivately examining Roman Wide Field Instrument (WFI) Advanced Scientific Data Format (ASDF) images
- *roman_datamodels* for opening Roman WFI ASDF files

In [ ]:
%matplotlib inline
from astropy.visualization import simple_norm
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

from jdaviz import Imviz
import roman_datamodels as rdm
import s3fs

## Introduction
In this tutorial, we will demonstrate how to visualize and explore Roman WFI image data arrays. We will separate this tutorial into two high-level components:
1. creating static images with world coordinate system (WCS) overlays; and
2. using the Jdaviz tool to interactively explore WFI image data.

For our tutorial, we will focus on how to visualize WFI Level 2 (L2) data contained within ASDF files. For WFI data, L2 indicates that the data have been processed to flag and/or correct for detector-level effects (e.g., saturation, classic non-linearity, etc.), and the pixels have been fitted up-the-ramp into a count rate image. Each L2 ASDF file contains a single WFI detector, thus a complete WFI exposure is made up of 18 L2 files. For more information on the contents of WFI L2 files, please see the RDox article on [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-Level2-CalibratedExposuresLevel2).

***

## Loading data
For both of our examples, we will use the same data file. The example of creating static images can be used on any `numpy.ndarray` object and the WCS axes may be optionally included with any `astropy.wcs.WCS` object. The Jdaviz example below is designed to work for Roman, but Jdaviz can also work with other data and file types. For the interested reader, a link to the Jdaviz documentation is provided in the Additional Resources section below.

A fuller explanation of how to load and work with Roman ASDF files is provided in the notebook tutorial Working with ASDF, and we assume that the reader has already consulted that tutorial. For now, we will read in the data using the `roman_datamodels` package.

In [ ]:
asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
fs = s3fs.S3FileSystem()

asdf_file_uri = asdf_dir_uri + 'ROMANISIM/DENSE_REGION/R0.5_DP0.5_PA0/r0000101001001001001_01101_0001_WFI16_cal.asdf'
with fs.open(asdf_file_uri, 'rb') as f:
    file = rdm.open(f)

## Static Image Display

### Plot an Image with Dynamic Scaling

Here we show how to use matplotlib and Astropy to plot the data array from our WFI image and scale the array automatically.

In [ ]:
# Set the image normalization. Here we use a log scale 
# with the minimum and maximum of the range based on 
# 99% of the pixels.
norm = simple_norm(file.data.value, 'asinh', percent=95)

fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.imshow(file.data.value, norm=norm, origin='lower')
ax.set_xlabel('X Science Axis (pixels)')
ax.set_ylabel('Y Science Axis (pixels)')
ax.set_title('romanisim Simulation WFI16')
plt.colorbar(sc, ax=ax)
plt.tight_layout();

We can see a bright, extended source in the lower portion of the image. Here we will isolate that and examine that region a little more closely. Looking at the output image above, we can see that it is located approximately between science Y coordinates 750 – 1250 and science X coordinates 1750 – 2250.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc = ax.imshow(file.data.value[750:1250, 1750:2250], norm=norm, origin='lower')
ax.set_xlabel('X Science Axis (pixels)')
ax.set_ylabel('Y Science Axis (pixels)')
ax.set_title('romanisim Simulation WFI16')
plt.colorbar(sc, ax=ax)
plt.tight_layout();

Now if we want to add information about the sky coordinates to the image rather than the detector pixel positions, we can do that using the `gwcs.wcs.WCS` object in the file metadata.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': file.meta.wcs})
sc = ax.imshow(file.data.value, norm=norm, origin='lower')
ax.grid(':', color='white')
plt.colorbar(sc, ax=ax)
ax.set_xlabel('Right Ascension (deg)')
ax.set_ylabel('Declination (deg)');

## Interactively Examine the Data Using Imviz

We can also use Imviz, the 2-D image viewer from the Jdaviz package, to visualize and explore the 2-D arrays contained within WFI L2 ASDF files. We highly recommend that users consult the [Imviz documentation](https://jdaviz.readthedocs.io/en/latest/imviz/index.html), which describes many of the features in Imviz in detail. In this tutorial, we will cover the basics to get you started.

**Note:** An enhancement to Cubeviz, the 3-D datacube visualization tool in Jdaviz, will soon allow interactive exploration of WFI L1 ramp cubes. This functionality is currently being tested and will be in a future Jdaviz release.

The first cell below will load Imviz and make a split panel to the right. You can control the height, or use a pop out window or an inline viewer. Please see the [display options](https://jdaviz.readthedocs.io/en/latest/display.html) documentation for more information.

In [ ]:
imviz = Imviz()
imviz.show('sidecar:split-right', height=1000)

Next, we load the image into Imviz. By default, for Roman WFI data, Imviz will only load the data array in the viewer to improve performance. Additional arrays (e.g., the data quality array) may be loaded using the `ext` optional argument. An example demonstrating how to load the data quality array is provided in a commented line in the following cell. See the RDox article on WFI [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-L2ScienceDataSpecifications) for more information on the arrays contained within WFI L2 ASDF files.

In [ ]:
asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
fs = s3fs.S3FileSystem()

asdf_file_uri = asdf_dir_uri + 'ROMANISIM/DENSE_REGION/R0.5_DP0.5_PA0/r0000101001001001001_01101_0001_WFI16_cal.asdf'
with fs.open(asdf_file_uri, 'rb') as f:
    file = rdm.open(f)
    imviz.load_data(file, data_label='WFI16_POS1')
    # imviz.load_data(file, ext='dq', data_label='WFI16 DQ')

The colormap, stretch, and scale limits may be adjusted interactively in the viewer by clicking the plot options icon (three vertically-stacked sliders to the far right of the teal bar above the image display, hereafter the "tool bar"), and expanding the "Plot Options" section. Note that there are additional options such as the minimum and maximum scale limits under the "More Stretch Options" expander.

If we know the settings that we want to apply, we can do so via the API like so:

In [ ]:
viewer = imviz.default_viewer
viewer.stretch_options
viewer.stretch = 'arcsinh'
viewer.set_colormap('Viridis')
viewer.cuts = (0, 40)

Now we will add a second image to the display, this time of the same field but dithered slightly, and we will link the two images by their WCS.

In [ ]:
asdf_file_uri = asdf_dir_uri + 'ROMANISIM/DENSE_REGION/R0.47_DP0.51_PA0/r0000101001001001001_01101_0002_WFI16_cal.asdf'
with fs.open(asdf_file_uri, 'rb') as f:
    file = rdm.open(f)
    imviz.load_data(file, data_label='WFI16_POS2')

viewer = imviz.default_viewer
viewer.stretch_options
viewer.stretch = 'arcsinh'
viewer.set_colormap('Viridis')
viewer.cuts = (0, 40)

Similar to our matplotlib demonstration of how to create a static image, let's investigate a region of interest and focus on the large extended source in the lower portion of the image. In this case, we know the galaxy has science pixel coordinates of (X, Y) ~ (1189, 737) pixels or sky coordinates of (RA, Dec) ~ (0.836, 0.598) degrees. This information can be determined from the catalog used to make the simulated products, or may alternatively have come from interactive examintion of the image or a source detection algorithm. Regardless of how we obtained the coordinates, we can specify the viewer be centered there using the API as shown below:

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord(ra=0.83608418, dec=0.5977862, unit=('deg', 'deg'))
viewer.center_on(sky)

# Center the image on given pixel coordinates.
# viewer.center_on((1189, 737))

We can also set the zoom level to better display the region around the extended source. The zoom level is set as:
* 1 means real-pixel-size.
* 2 means zoomed in by a factor of 2.
* 0.5 means zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image width into display. In this case, we will set the zoom level to 0.4 so we can see the extended source in both images and blink between them. To blink the images, place the cursor over the viewer and press the "b" key on your keyboard.

In [ ]:
viewer.zoom_level = 0.4

As you can see when blinking the images, the sources move between images because the simulated field of view has changed position slightly. If we link the images by their WCS information, we see that the sources now remain fixed, but the parts of the sky observed are slightly different.

**Note:** Roman WFI ASDF files use a Generalized World Coordinate System (gwcs) object in Python to store the WCS transformation. The transformation is only well-defined within a bounding box, and moving outside that bounding box produces unexpected behavior, particularly due to the polynomial terms in the transformation between pixel and sky coordinates. The current version of Imviz has difficulty with this, so programmatically setting the position and zoom after linking the images by WCS may produce undesirable results.

In [ ]:
orientation = imviz.plugins['Orientation']
orientation.link_type = 'WCS'

You can also save the current view to a PNG file on disk:

In [ ]:
viewer.save('my_image.png')

For more advanced use cases such as interactive aperture photometry or line profiles, please consult the [Imviz documentation](https://jdaviz.readthedocs.io/en/latest/imviz/index.html).

## Aditional Resources

- [Imviz Documentation](https://jdaviz.readthedocs.io/en/latest/imviz/index.html)
- [Additional JDaviz Notebooks](https://github.com/spacetelescope/jdaviz/tree/main/notebooks)
- [RDox WFI Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products#DataLevelsandProducts-L2ScienceDataSpecifications)

## About this notebook
**Author:** Tyler Desjardins, Brett Morris  
**Updated On:** 2024-05-03

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 